In [ ]:
# This script was created to convert .csv files of wind data from https://download.synopticdata.com/ (MesoWest)
# It will take the raw data and return a new file that has Sustained Wind Speed, Gust Factor, Year, Month, and Day.
# The Gust Factor is calculated using the sustained wind speed and gust wind speed values.

# Coded by: Brennan Dettmann
# Date: 6/5/2020

In [1]:
# imports all of the necessary modules

#import sys
import csv
import os
import pandas as pd

In [ ]:
#df = pd.read_csv('C:/Users/brenn/Desktop/Juneau Internship/Data/LAND/Final Datasets/BLTA2_kts.csv')

In [2]:
filename = input('Input Filename: ') # make sure to put in format {station}_{units} ex. PAGN_kts
station = filename.split("_")[0]
units = filename.split("_")[1]

df = pd.read_csv(f'{station}_{units}.csv', sep=',', header=[11])
pulledFile = open(f'{filename}.csv')
reader = csv.reader(pulledFile, delimiter = ",")

allDir = open(f'/home/bdettman/Juneau_Internship/Wind-Gust-Climatology/sorteddata/{station}_All.csv', 'a+')
allDirwriter = csv.writer(allDir, delimiter =',', quotechar='"', quoting = csv.QUOTE_ALL)
allDirwriter.writerow(["Sustained Wind Speed (kts)", "Gust Factor", "Year", "Month", "Day"])

Input Filename:  BLTA2_kts


65

In [3]:
df

,Unnamed: 0,Unnamed: 1,knots,Degrees,knots.1,Code
0,BLTA2,2010-08-24T18:25:00Z,0.00,246.0,1.75,WSW
1,BLTA2,2010-08-24T20:25:00Z,1.75,269.0,4.33,W
2,BLTA2,2010-08-24T21:25:00Z,0.87,260.0,2.60,W
3,BLTA2,2010-08-24T22:25:00Z,1.75,262.0,5.21,W
4,BLTA2,2010-08-24T23:25:00Z,0.87,247.0,2.60,WSW
...,...,...,...,...,...,...
42307,BLTA2,2020-04-30T19:25:00Z,5.03,129.9,8.51,SE
42308,BLTA2,2020-04-30T20:25:00Z,4.88,141.8,8.34,SE
42309,BLTA2,2020-04-30T21:25:00Z,4.78,155.4,9.29,SSE
42310,BLTA2,2020-04-30T22:25:00Z,4.33,144.0,8.09,SE


In [4]:
def directionconverter(drxn):
#if len(drxn.strip()) != 3: return drxn.strip()	
#else:
        drxn = int(drxn)
        if drxn >= 340 or drxn < 30: return "N"
        elif drxn < 70: return "NE"
        elif drxn < 120: return "E"
        elif drxn < 160: return "SE"
        elif drxn < 210: return "S"
        elif drxn < 250: return "SW"
        elif drxn < 300: return "W"
        elif drxn < 340: return "NW"
        else: return drxn.strip()

In [5]:
#df = df.fillna(0)
#df.IntDegrees = df.Degrees.astype(int)
for i in range(10):  # testing the loop
    print(directionconverter(df.Degrees[i]))

SW
W
W
W
SW
NE
N
W
N
N


In [6]:
df = df.fillna(0)

In [8]:
for row in reader:
    if len(row) == 6:
        if len(row[3].strip()) in range(1,4):
            for i in range(42311):
                windDir = directionconverter(df.Degrees[i])
                if any(num in row[1] for num in ["1", "2", "3", "4", "5", "6", "7", "8", "9"]):
                    if units == "mph":
                        windSpeed = float(row[2])*.87
                    if units == "knots" or units == "kts":
                        windSpeed = float(row[2])
                    windGust = float(row[4])
                    gustMultiplier = windGust/windSpeed
                    year = row[1][0:4]
                    month = row[1][5:7]
                    day = row[1][8:10]
                    allDirwriter.writerow([windSpeed, gustMultiplier, year, month, day])
                    oneDir=open((f'/home/bdettman/Juneau_Internship/sorteddata/{station}_{windDir}.csv'),'a+')
                    oneDirwriter = csv.writer(oneDir, delimiter =',', quotechar='"', quoting = csv.QUOTE_ALL)
                    if os.path.getsize(f'/home/bdettman/Juneau_Internship/sorteddata/{station}_{windDir}.csv') == 0:
                        oneDirwriter.writerow(["Sustained Wind Speed (kts)", "Gust Factor", "Year", "Month", "Day"])
                    oneDirwriter.writerow([windSpeed, gustMultiplier, year, month, day])
                    oneDir.close()

allDir.close()
pulledFile.close()

FileNotFoundError: [Errno 2] No such file or directory: '/home/bdettman/Juneau_Internship/sorteddata/BLTA2_SW.csv'